In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix,classification_report, precision_score,recall_score,f1_score
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from matplotlib import style
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline
style.use('ggplot')

df = pd.read_excel('PL_2019-2020.xlsx')
df

,SEASON_CD,SEASON_NAME,PLOT_CLUSTER_CD,CLUSTER_NAME,PLOT_VILLAGE_CD,VILLAGE_NAME,FARMER_ID,F_NAME,PLOT_NO,AG_FLAG,...,DIVERTED_AREA,DISPOSAL_AREA,BOOKED_AREA,NON_BOOKED_AREA,SUPPLIED_TONNS_1,YIELD,AGE_IN_DAYS,GBL_PLANT_ID,GBL_KHATE_ID,GBL_PLOT_ID
0,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,13.0,NaN,...,0.0,0.0,4.0,0.0,187.763,46.941,426,914297,11,12
1,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,4.0,NaN,...,0.0,0.0,5.0,0.0,99.020,19.804,361,984477,10,9
2,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,5.0,NaN,...,0.0,0.0,5.0,0.0,40.679,8.136,347,985669,10,10
3,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,7.0,NaN,...,6.0,0.0,6.0,0.0,0.000,NaN,524,903118,19,26
4,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100004,VENKAPPA SHASAPPA KAMADINNI,1.0,NaN,...,0.0,0.0,5.0,0.0,57.281,11.456,387,917920,20,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53420,25,2019-2020,28,LOKAPUR,1453,KAMATAGI,365911,KARIYAPPA HUCHAPPA HUGGI,2.0,NaN,...,0.0,2.0,0.0,2.0,0.000,NaN,65,979755,238108,327730
53421,25,2019-2020,28,LOKAPUR,1453,KAMATAGI,369510,SAKRAPPA YALLAPPA HAVOJI,1.0,NaN,...,0.0,0.0,1.0,0.0,21.206,21.206,404,1011696,246928,337841
53422,25,2019-2020,28,LOKAPUR,1468,SHIROL (GADAG),371252,SAVITRI MUDAKANAGOUDA TIRAKANAGOUDR,1.0,NaN,...,0.0,0.0,1.0,0.0,23.849,23.849,386,1018000,250026,341175
53423,25,2019-2020,28,LOKAPUR,1468,SHIROL (GADAG),371252,SAVITRI MUDAKANAGOUDA TIRAKANAGOUDR,2.0,NaN,...,0.0,0.0,1.0,0.0,21.810,21.810,392,1018001,250026,341176


In [2]:
CV_NAMES=['86032','91010','M 265','SNK-09293','PI 00-1110','CO VSI 08005']
df1=df[(df.CV_NAME.isin (CV_NAMES)) & (((df.DISPOSAL_DATE - datetime.datetime(2018,1,1)).dt.days) > 0)].copy()
df1

,SEASON_CD,SEASON_NAME,PLOT_CLUSTER_CD,CLUSTER_NAME,PLOT_VILLAGE_CD,VILLAGE_NAME,FARMER_ID,F_NAME,PLOT_NO,AG_FLAG,...,DIVERTED_AREA,DISPOSAL_AREA,BOOKED_AREA,NON_BOOKED_AREA,SUPPLIED_TONNS_1,YIELD,AGE_IN_DAYS,GBL_PLANT_ID,GBL_KHATE_ID,GBL_PLOT_ID
0,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,13.0,NaN,...,0.0,0.0,4.0,0.0,187.763,46.941,426,914297,11,12
1,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,4.0,NaN,...,0.0,0.0,5.0,0.0,99.020,19.804,361,984477,10,9
2,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,5.0,NaN,...,0.0,0.0,5.0,0.0,40.679,8.136,347,985669,10,10
3,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,7.0,NaN,...,6.0,0.0,6.0,0.0,0.000,NaN,524,903118,19,26
4,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100004,VENKAPPA SHASAPPA KAMADINNI,1.0,NaN,...,0.0,0.0,5.0,0.0,57.281,11.456,387,917920,20,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53420,25,2019-2020,28,LOKAPUR,1453,KAMATAGI,365911,KARIYAPPA HUCHAPPA HUGGI,2.0,NaN,...,0.0,2.0,0.0,2.0,0.000,NaN,65,979755,238108,327730
53421,25,2019-2020,28,LOKAPUR,1453,KAMATAGI,369510,SAKRAPPA YALLAPPA HAVOJI,1.0,NaN,...,0.0,0.0,1.0,0.0,21.206,21.206,404,1011696,246928,337841
53422,25,2019-2020,28,LOKAPUR,1468,SHIROL (GADAG),371252,SAVITRI MUDAKANAGOUDA TIRAKANAGOUDR,1.0,NaN,...,0.0,0.0,1.0,0.0,23.849,23.849,386,1018000,250026,341175
53423,25,2019-2020,28,LOKAPUR,1468,SHIROL (GADAG),371252,SAVITRI MUDAKANAGOUDA TIRAKANAGOUDR,2.0,NaN,...,0.0,0.0,1.0,0.0,21.810,21.810,392,1018001,250026,341176


In [3]:
df1['DISPOSAL_TIME']=((df1.DISPOSAL_DATE - datetime.datetime(2018,1,1)).dt.days)
df1

,SEASON_CD,SEASON_NAME,PLOT_CLUSTER_CD,CLUSTER_NAME,PLOT_VILLAGE_CD,VILLAGE_NAME,FARMER_ID,F_NAME,PLOT_NO,AG_FLAG,...,DISPOSAL_AREA,BOOKED_AREA,NON_BOOKED_AREA,SUPPLIED_TONNS_1,YIELD,AGE_IN_DAYS,GBL_PLANT_ID,GBL_KHATE_ID,GBL_PLOT_ID,DISPOSAL_TIME
0,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,13.0,NaN,...,0.0,4.0,0.0,187.763,46.941,426,914297,11,12,762
1,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,4.0,NaN,...,0.0,5.0,0.0,99.020,19.804,361,984477,10,9,755
2,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,5.0,NaN,...,0.0,5.0,0.0,40.679,8.136,347,985669,10,10,769
3,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,7.0,NaN,...,0.0,6.0,0.0,0.000,NaN,524,903118,19,26,704
4,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100004,VENKAPPA SHASAPPA KAMADINNI,1.0,NaN,...,0.0,5.0,0.0,57.281,11.456,387,917920,20,27,766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53420,25,2019-2020,28,LOKAPUR,1453,KAMATAGI,365911,KARIYAPPA HUCHAPPA HUGGI,2.0,NaN,...,2.0,0.0,2.0,0.000,NaN,65,979755,238108,327730,479
53421,25,2019-2020,28,LOKAPUR,1453,KAMATAGI,369510,SAKRAPPA YALLAPPA HAVOJI,1.0,NaN,...,0.0,1.0,0.0,21.206,21.206,404,1011696,246928,337841,749
53422,25,2019-2020,28,LOKAPUR,1468,SHIROL (GADAG),371252,SAVITRI MUDAKANAGOUDA TIRAKANAGOUDR,1.0,NaN,...,0.0,1.0,0.0,23.849,23.849,386,1018000,250026,341175,792
53423,25,2019-2020,28,LOKAPUR,1468,SHIROL (GADAG),371252,SAVITRI MUDAKANAGOUDA TIRAKANAGOUDR,2.0,NaN,...,0.0,1.0,0.0,21.810,21.810,392,1018001,250026,341176,792


In [4]:
X1=df1[["CV_NAME","CLUSTER_NAME","PLANT_AREA","CROP_TYPE_NAME","AGE_IN_DAYS","DISPOSAL_TIME"]].copy()
Y1=df1["DIVERSION_CENTER_NAME"].copy()
X1,Y1

(      CV_NAME  CLUSTER_NAME  PLANT_AREA CROP_TYPE_NAME  AGE_IN_DAYS  \
 0       91010  FACTORY EAST         4.0             R1          426   
 1       91010  FACTORY EAST         5.0             R2          361   
 2       91010  FACTORY EAST         5.0             R2          347   
 3       86032  FACTORY EAST         6.0             PL          524   
 4       M 265  FACTORY EAST         5.0             R1          387   
 ...       ...           ...         ...            ...          ...   
 53420   86032       LOKAPUR         2.0             R1           65   
 53421   86032       LOKAPUR         1.0             PL          404   
 53422   91010       LOKAPUR         1.0             PL          386   
 53423   91010       LOKAPUR         1.0             PL          392   
 53424   86032       LOKAPUR         1.0             PL          359   
 
        DISPOSAL_TIME  
 0                762  
 1                755  
 2                769  
 3                704  
 4            

In [5]:
for i in Y1.index:
    Y1[i]=(1 if (Y1[i]=="GSM_HARVEST") else 0)
Y1_new=Y1.astype('int')

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

features = ["CLUSTER_NAME","CROP_TYPE_NAME","CV_NAME"]
one = OneHotEncoder(sparse=False)
transformer = ColumnTransformer([("one",
                                  one,
                                  features)],
                                  remainder="passthrough")
X1_new = pd.DataFrame(transformer.fit_transform(X1))
X1_new,Y1_new

(        0    1    2    3    4    5    6    7    8    9   ...   22   23   24  \
 0      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  1.0   
 1      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  1.0   
 2      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  1.0   
 3      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  1.0  0.0   
 4      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 ...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
 52418  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  1.0  0.0   
 52419  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  1.0  0.0   
 52420  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0  1.0   
 52421  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0  1.0   
 52422  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  1.0  0.0   
 
         25   26   27   28   29     30

In [6]:
X1train,X1test,Y1train,Y1test =train_test_split(X1_new,Y1_new,test_size=0.2)
def score(model):
    model.fit(X1train,Y1train)
    print(model.score(X1test,Y1test))

In [6]:
hgbc = HistGradientBoostingClassifier()
xgboost = XGBClassifier(use_label_encoder=False)
catboost = CatBoostClassifier()
lgbm = LGBMClassifier()
rf = RandomForestClassifier()
kn = KNeighborsClassifier()
lr = LogisticRegression(max_iter=4000)
rc = RidgeClassifier()
svc = SVC()
bc = BaggingClassifier()
gbc = GradientBoostingClassifier()

In [8]:
score(rf)
score(lr)
score(kn)
score(rc)
score(svc)
score(bc)
score(gbc)
score(hgbc)
score(lgbm)
score(xgboost)
score(catboost)

0.9053886504530282
0.8568431092036243
0.8682880305197902
0.805722460658083
0.8253695755841679
0.8977587029089175
0.8965188364329996
0.9070100143061517
0.9087267525035765
[10:32:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9110157367668097
Learning rate set to 0.050791
0:	learn: 0.6260258	total: 180ms	remaining: 3m
1:	learn: 0.5713986	total: 191ms	remaining: 1m 35s
2:	learn: 0.5260775	total: 203ms	remaining: 1m 7s
3:	learn: 0.4895107	total: 213ms	remaining: 53.1s
4:	learn: 0.4633455	total: 224ms	remaining: 44.5s
5:	learn: 0.4396007	total: 235ms	remaining: 39s
6:	learn: 0.4219981	total: 246ms	remaining: 34.9s
7:	learn: 0.4052291	total: 256ms	remaining: 31.8s
8:	learn: 0.3945680	total: 267ms	remaining: 29.4s
9:	learn: 0.3839495	total

164:	learn: 0.2407734	total: 1.93s	remaining: 9.74s
165:	learn: 0.2406508	total: 1.94s	remaining: 9.73s
166:	learn: 0.2405357	total: 1.95s	remaining: 9.71s
167:	learn: 0.2403592	total: 1.96s	remaining: 9.69s
168:	learn: 0.2402140	total: 1.97s	remaining: 9.68s
169:	learn: 0.2400237	total: 1.98s	remaining: 9.66s
170:	learn: 0.2398797	total: 1.99s	remaining: 9.64s
171:	learn: 0.2397795	total: 2s	remaining: 9.63s
172:	learn: 0.2396085	total: 2.01s	remaining: 9.61s
173:	learn: 0.2394368	total: 2.02s	remaining: 9.59s
174:	learn: 0.2392333	total: 2.03s	remaining: 9.57s
175:	learn: 0.2390495	total: 2.04s	remaining: 9.56s
176:	learn: 0.2389015	total: 2.05s	remaining: 9.55s
177:	learn: 0.2387477	total: 2.06s	remaining: 9.53s
178:	learn: 0.2386344	total: 2.08s	remaining: 9.52s
179:	learn: 0.2384483	total: 2.09s	remaining: 9.51s
180:	learn: 0.2383017	total: 2.1s	remaining: 9.49s
181:	learn: 0.2381807	total: 2.11s	remaining: 9.48s
182:	learn: 0.2379916	total: 2.12s	remaining: 9.46s
183:	learn: 0.23

330:	learn: 0.2204149	total: 3.68s	remaining: 7.44s
331:	learn: 0.2202490	total: 3.69s	remaining: 7.43s
332:	learn: 0.2201952	total: 3.71s	remaining: 7.42s
333:	learn: 0.2201205	total: 3.72s	remaining: 7.41s
334:	learn: 0.2200043	total: 3.73s	remaining: 7.4s
335:	learn: 0.2199050	total: 3.74s	remaining: 7.39s
336:	learn: 0.2198076	total: 3.75s	remaining: 7.38s
337:	learn: 0.2197274	total: 3.76s	remaining: 7.36s
338:	learn: 0.2196208	total: 3.77s	remaining: 7.35s
339:	learn: 0.2195420	total: 3.78s	remaining: 7.33s
340:	learn: 0.2194632	total: 3.79s	remaining: 7.32s
341:	learn: 0.2193860	total: 3.8s	remaining: 7.31s
342:	learn: 0.2193024	total: 3.81s	remaining: 7.3s
343:	learn: 0.2191967	total: 3.82s	remaining: 7.29s
344:	learn: 0.2191229	total: 3.83s	remaining: 7.27s
345:	learn: 0.2190416	total: 3.84s	remaining: 7.26s
346:	learn: 0.2190047	total: 3.85s	remaining: 7.25s
347:	learn: 0.2189339	total: 3.86s	remaining: 7.23s
348:	learn: 0.2188302	total: 3.87s	remaining: 7.22s
349:	learn: 0.2

499:	learn: 0.2087057	total: 5.45s	remaining: 5.45s
500:	learn: 0.2086427	total: 5.46s	remaining: 5.43s
501:	learn: 0.2085411	total: 5.47s	remaining: 5.42s
502:	learn: 0.2084623	total: 5.48s	remaining: 5.42s
503:	learn: 0.2083274	total: 5.49s	remaining: 5.41s
504:	learn: 0.2082616	total: 5.5s	remaining: 5.39s
505:	learn: 0.2082355	total: 5.51s	remaining: 5.38s
506:	learn: 0.2081758	total: 5.53s	remaining: 5.37s
507:	learn: 0.2081002	total: 5.54s	remaining: 5.36s
508:	learn: 0.2080461	total: 5.55s	remaining: 5.35s
509:	learn: 0.2079954	total: 5.56s	remaining: 5.34s
510:	learn: 0.2079275	total: 5.57s	remaining: 5.33s
511:	learn: 0.2078591	total: 5.58s	remaining: 5.32s
512:	learn: 0.2078171	total: 5.59s	remaining: 5.31s
513:	learn: 0.2077768	total: 5.6s	remaining: 5.3s
514:	learn: 0.2077485	total: 5.61s	remaining: 5.29s
515:	learn: 0.2076845	total: 5.62s	remaining: 5.27s
516:	learn: 0.2076332	total: 5.63s	remaining: 5.26s
517:	learn: 0.2075788	total: 5.64s	remaining: 5.25s
518:	learn: 0.2

661:	learn: 0.1998536	total: 7.13s	remaining: 3.64s
662:	learn: 0.1997911	total: 7.14s	remaining: 3.63s
663:	learn: 0.1997335	total: 7.15s	remaining: 3.62s
664:	learn: 0.1996723	total: 7.16s	remaining: 3.61s
665:	learn: 0.1996045	total: 7.17s	remaining: 3.6s
666:	learn: 0.1995491	total: 7.18s	remaining: 3.59s
667:	learn: 0.1994528	total: 7.19s	remaining: 3.58s
668:	learn: 0.1993941	total: 7.2s	remaining: 3.56s
669:	learn: 0.1993154	total: 7.21s	remaining: 3.55s
670:	learn: 0.1992650	total: 7.22s	remaining: 3.54s
671:	learn: 0.1992015	total: 7.24s	remaining: 3.53s
672:	learn: 0.1991377	total: 7.25s	remaining: 3.52s
673:	learn: 0.1990868	total: 7.25s	remaining: 3.51s
674:	learn: 0.1990438	total: 7.27s	remaining: 3.5s
675:	learn: 0.1989985	total: 7.28s	remaining: 3.49s
676:	learn: 0.1989381	total: 7.29s	remaining: 3.48s
677:	learn: 0.1988865	total: 7.3s	remaining: 3.47s
678:	learn: 0.1988406	total: 7.31s	remaining: 3.46s
679:	learn: 0.1987800	total: 7.32s	remaining: 3.44s
680:	learn: 0.19

827:	learn: 0.1926741	total: 8.87s	remaining: 1.84s
828:	learn: 0.1926386	total: 8.88s	remaining: 1.83s
829:	learn: 0.1926233	total: 8.89s	remaining: 1.82s
830:	learn: 0.1925842	total: 8.9s	remaining: 1.81s
831:	learn: 0.1925403	total: 8.91s	remaining: 1.8s
832:	learn: 0.1924830	total: 8.92s	remaining: 1.79s
833:	learn: 0.1924391	total: 8.93s	remaining: 1.78s
834:	learn: 0.1923956	total: 8.95s	remaining: 1.77s
835:	learn: 0.1923508	total: 8.95s	remaining: 1.76s
836:	learn: 0.1922993	total: 8.96s	remaining: 1.75s
837:	learn: 0.1922744	total: 8.97s	remaining: 1.74s
838:	learn: 0.1922438	total: 8.98s	remaining: 1.72s
839:	learn: 0.1921998	total: 9s	remaining: 1.71s
840:	learn: 0.1921571	total: 9.01s	remaining: 1.7s
841:	learn: 0.1920956	total: 9.02s	remaining: 1.69s
842:	learn: 0.1920340	total: 9.03s	remaining: 1.68s
843:	learn: 0.1920035	total: 9.04s	remaining: 1.67s
844:	learn: 0.1919671	total: 9.05s	remaining: 1.66s
845:	learn: 0.1919153	total: 9.06s	remaining: 1.65s
846:	learn: 0.1918

997:	learn: 0.1862936	total: 10.7s	remaining: 21.4ms
998:	learn: 0.1862920	total: 10.7s	remaining: 10.7ms
999:	learn: 0.1862680	total: 10.7s	remaining: 0us
0.9133047210300429


In [7]:
df2=df1.sort_values('DISPOSAL_TIME').copy()
df2

,SEASON_CD,SEASON_NAME,PLOT_CLUSTER_CD,CLUSTER_NAME,PLOT_VILLAGE_CD,VILLAGE_NAME,FARMER_ID,F_NAME,PLOT_NO,AG_FLAG,...,DISPOSAL_AREA,BOOKED_AREA,NON_BOOKED_AREA,SUPPLIED_TONNS_1,YIELD,AGE_IN_DAYS,GBL_PLANT_ID,GBL_KHATE_ID,GBL_PLOT_ID,DISPOSAL_TIME
35122,25,2019-2020,9,MUDHOL-SE,312,CHIKKUR,109725,ASHOK RAMAPPA GALAGALI,3.0,NaN,...,3.0,3.0,0.0,0.0,NaN,-217,993980,19971,37526,4
47629,25,2019-2020,20,SHIRAGUPPI,909,JAMAKHANDI-N,144756,MURAGEPPA BASAPPA KOUJALAGI,92.0,NaN,...,3.0,3.0,0.0,0.0,NaN,-335,969004,225147,311815,14
12481,25,2019-2020,2,FACTORY-SW,900,KAPPALAGUDDI-N,364757,ALLAPPA SHANMUK ANGADI,1.0,NaN,...,1.0,1.0,0.0,0.0,NaN,-335,978559,232595,320753,16
825,25,2019-2020,1,FACTORY EAST,4,BELAGALI,100820,SHIDAGIREPPA MAHALINGAPP BELAGALI,6.0,NaN,...,2.5,2.5,0.0,0.0,NaN,-309,984981,1424,2676,19
9327,25,2019-2020,2,FACTORY-SW,18,SHIVAPUR,301042,MAHADEV SHIVABASU SILANNAVAR,1.0,NaN,...,2.0,2.0,0.0,0.0,NaN,-310,971149,146630,218730,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1330,25,2019-2020,1,FACTORY EAST,4,BELAGALI,349833,SATAWWA CHANNAYYA CHATTI,3.0,NaN,...,0.0,1.5,0.0,0.0,NaN,364,1069689,192604,273245,870
7946,25,2019-2020,2,FACTORY-SW,16,MARAKUDI,151027,VITHAL TULAJAPPA MALAGAR,4.0,NaN,...,0.0,1.7,0.0,0.0,NaN,360,1074192,250324,341527,872
7883,25,2019-2020,2,FACTORY-SW,16,MARAKUDI,133379,BASAPPA KALLAPPA KULIGOD,1.0,NaN,...,0.0,1.5,0.0,0.0,NaN,368,1070306,80333,126982,872
12058,25,2019-2020,2,FACTORY-SW,900,KAPPALAGUDDI-N,143662,SADAPPA TUKKAPPA BHADRASHETTI,4.0,NaN,...,0.0,1.5,0.0,0.0,NaN,380,1075021,103873,160659,872


In [8]:
#'86032','91010','M 265','SNK-09293','PI 00-1110','CO VSI 08005'
df2['DISPO_86032']=0
df2['DISPO_91010']=0
df2['DISPO_M 265']=0
df2['DISPO_SNK-09293']=0
df2['DISPO_PI 00-1110']=0
df2['DISPO_CO VSI 08005']=0
df2['DISPO_CURR']=0
df2

,SEASON_CD,SEASON_NAME,PLOT_CLUSTER_CD,CLUSTER_NAME,PLOT_VILLAGE_CD,VILLAGE_NAME,FARMER_ID,F_NAME,PLOT_NO,AG_FLAG,...,GBL_KHATE_ID,GBL_PLOT_ID,DISPOSAL_TIME,DISPO_86032,DISPO_91010,DISPO_M 265,DISPO_SNK-09293,DISPO_PI 00-1110,DISPO_CO VSI 08005,DISPO_CURR
35122,25,2019-2020,9,MUDHOL-SE,312,CHIKKUR,109725,ASHOK RAMAPPA GALAGALI,3.0,NaN,...,19971,37526,4,0,0,0,0,0,0,0
47629,25,2019-2020,20,SHIRAGUPPI,909,JAMAKHANDI-N,144756,MURAGEPPA BASAPPA KOUJALAGI,92.0,NaN,...,225147,311815,14,0,0,0,0,0,0,0
12481,25,2019-2020,2,FACTORY-SW,900,KAPPALAGUDDI-N,364757,ALLAPPA SHANMUK ANGADI,1.0,NaN,...,232595,320753,16,0,0,0,0,0,0,0
825,25,2019-2020,1,FACTORY EAST,4,BELAGALI,100820,SHIDAGIREPPA MAHALINGAPP BELAGALI,6.0,NaN,...,1424,2676,19,0,0,0,0,0,0,0
9327,25,2019-2020,2,FACTORY-SW,18,SHIVAPUR,301042,MAHADEV SHIVABASU SILANNAVAR,1.0,NaN,...,146630,218730,40,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1330,25,2019-2020,1,FACTORY EAST,4,BELAGALI,349833,SATAWWA CHANNAYYA CHATTI,3.0,NaN,...,192604,273245,870,0,0,0,0,0,0,0
7946,25,2019-2020,2,FACTORY-SW,16,MARAKUDI,151027,VITHAL TULAJAPPA MALAGAR,4.0,NaN,...,250324,341527,872,0,0,0,0,0,0,0
7883,25,2019-2020,2,FACTORY-SW,16,MARAKUDI,133379,BASAPPA KALLAPPA KULIGOD,1.0,NaN,...,80333,126982,872,0,0,0,0,0,0,0
12058,25,2019-2020,2,FACTORY-SW,900,KAPPALAGUDDI-N,143662,SADAPPA TUKKAPPA BHADRASHETTI,4.0,NaN,...,103873,160659,872,0,0,0,0,0,0,0


In [9]:
df2['DISPO_86032']= df.DISPOSAL_TONNS.where(cond=(df2.CV_NAME == '86032'),other=df2.DISPO_86032)
df2['DISPO_91010']= df.DISPOSAL_TONNS.where(cond=(df2.CV_NAME == '91010'),other=df2.DISPO_91010)
df2['DISPO_M 265']= df.DISPOSAL_TONNS.where(cond=(df2.CV_NAME == 'M 265'),other=df2['DISPO_M 265'])
df2['DISPO_SNK-09293']= df.DISPOSAL_TONNS.where(cond=(df2.CV_NAME == 'SNK-09293'),other=df2['DISPO_SNK-09293'])
df2['DISPO_PI 00-1110']= df.DISPOSAL_TONNS.where(cond=(df2.CV_NAME == 'PI 00-1110'),other=df2['DISPO_PI 00-1110'])
df2['DISPO_CO VSI 08005']= df.DISPOSAL_TONNS.where(cond=(df2.CV_NAME == 'CO VSI 08005'),other=df2['DISPO_CO VSI 08005'])
df2[:40]

,SEASON_CD,SEASON_NAME,PLOT_CLUSTER_CD,CLUSTER_NAME,PLOT_VILLAGE_CD,VILLAGE_NAME,FARMER_ID,F_NAME,PLOT_NO,AG_FLAG,...,GBL_KHATE_ID,GBL_PLOT_ID,DISPOSAL_TIME,DISPO_86032,DISPO_91010,DISPO_M 265,DISPO_SNK-09293,DISPO_PI 00-1110,DISPO_CO VSI 08005,DISPO_CURR
35122,25,2019-2020,9,MUDHOL-SE,312,CHIKKUR,109725,ASHOK RAMAPPA GALAGALI,3.0,NaN,...,19971,37526,4,0.0,0.0,0.0,0.0,0.0,0.0,0
47629,25,2019-2020,20,SHIRAGUPPI,909,JAMAKHANDI-N,144756,MURAGEPPA BASAPPA KOUJALAGI,92.0,NaN,...,225147,311815,14,0.0,0.0,0.0,0.0,0.0,0.0,0
12481,25,2019-2020,2,FACTORY-SW,900,KAPPALAGUDDI-N,364757,ALLAPPA SHANMUK ANGADI,1.0,NaN,...,232595,320753,16,0.0,0.0,0.0,0.0,0.0,0.0,0
825,25,2019-2020,1,FACTORY EAST,4,BELAGALI,100820,SHIDAGIREPPA MAHALINGAPP BELAGALI,6.0,NaN,...,1424,2676,19,0.0,0.0,0.0,0.0,0.0,0.0,0
9327,25,2019-2020,2,FACTORY-SW,18,SHIVAPUR,301042,MAHADEV SHIVABASU SILANNAVAR,1.0,NaN,...,146630,218730,40,0.0,0.0,0.0,0.0,0.0,0.0,0
885,25,2019-2020,1,FACTORY EAST,4,BELAGALI,101034,CHANNAPPA YALLAPPA RODDANAVAR,3.0,NaN,...,1749,3205,40,0.0,0.0,0.0,0.0,0.0,0.0,0
9325,25,2019-2020,2,FACTORY-SW,18,SHIVAPUR,301041,BASAPPA IRAPPA SILANNAVAR,2.0,NaN,...,146629,218728,40,0.0,0.0,0.0,0.0,0.0,0.0,0
40375,25,2019-2020,18,TERADAL,883,KALATIPPI-N,134038,SANGAPPA SHIVAPPA PATIL,1.0,NaN,...,81910,129153,46,0.0,0.0,0.0,0.0,0.0,0.0,0
908,25,2019-2020,1,FACTORY EAST,4,BELAGALI,101114,BASALINGAPPA BASAPPA KURABETT,1.0,NaN,...,1876,3409,50,0.0,0.0,0.0,0.0,0.0,0.0,0
1102,25,2019-2020,1,FACTORY EAST,4,BELAGALI,155143,YAMANAPPA MAHADEVAPPA PASCHAPUR,5.0,NaN,...,124347,190303,50,0.0,0.0,0.0,0.0,0.0,0.0,0


In [10]:
df2['DISPO_86032']=df2['DISPO_86032'].cumsum()
df2['DISPO_91010']=df2['DISPO_91010'].cumsum()
df2['DISPO_M 265']=df2['DISPO_M 265'].cumsum()
df2['DISPO_SNK-09293']=df2['DISPO_SNK-09293'].cumsum()
df2['DISPO_PI 00-1110']=df2['DISPO_PI 00-1110'].cumsum()
df2['DISPO_CO VSI 08005']=df2['DISPO_CO VSI 08005'].cumsum()
df2

,SEASON_CD,SEASON_NAME,PLOT_CLUSTER_CD,CLUSTER_NAME,PLOT_VILLAGE_CD,VILLAGE_NAME,FARMER_ID,F_NAME,PLOT_NO,AG_FLAG,...,GBL_KHATE_ID,GBL_PLOT_ID,DISPOSAL_TIME,DISPO_86032,DISPO_91010,DISPO_M 265,DISPO_SNK-09293,DISPO_PI 00-1110,DISPO_CO VSI 08005,DISPO_CURR
35122,25,2019-2020,9,MUDHOL-SE,312,CHIKKUR,109725,ASHOK RAMAPPA GALAGALI,3.0,NaN,...,19971,37526,4,0.000,0.000,0.00,0.00,0.000,0.000,0
47629,25,2019-2020,20,SHIRAGUPPI,909,JAMAKHANDI-N,144756,MURAGEPPA BASAPPA KOUJALAGI,92.0,NaN,...,225147,311815,14,0.000,0.000,0.00,0.00,0.000,0.000,0
12481,25,2019-2020,2,FACTORY-SW,900,KAPPALAGUDDI-N,364757,ALLAPPA SHANMUK ANGADI,1.0,NaN,...,232595,320753,16,0.000,0.000,0.00,0.00,0.000,0.000,0
825,25,2019-2020,1,FACTORY EAST,4,BELAGALI,100820,SHIDAGIREPPA MAHALINGAPP BELAGALI,6.0,NaN,...,1424,2676,19,0.000,0.000,0.00,0.00,0.000,0.000,0
9327,25,2019-2020,2,FACTORY-SW,18,SHIVAPUR,301042,MAHADEV SHIVABASU SILANNAVAR,1.0,NaN,...,146630,218730,40,0.000,0.000,0.00,0.00,0.000,0.000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1330,25,2019-2020,1,FACTORY EAST,4,BELAGALI,349833,SATAWWA CHANNAYYA CHATTI,3.0,NaN,...,192604,273245,870,1549354.041,268549.322,101906.02,55498.17,17502.654,9832.638,0
7946,25,2019-2020,2,FACTORY-SW,16,MARAKUDI,151027,VITHAL TULAJAPPA MALAGAR,4.0,NaN,...,250324,341527,872,1549378.621,268549.322,101906.02,55498.17,17502.654,9832.638,0
7883,25,2019-2020,2,FACTORY-SW,16,MARAKUDI,133379,BASAPPA KALLAPPA KULIGOD,1.0,NaN,...,80333,126982,872,1549456.526,268549.322,101906.02,55498.17,17502.654,9832.638,0
12058,25,2019-2020,2,FACTORY-SW,900,KAPPALAGUDDI-N,143662,SADAPPA TUKKAPPA BHADRASHETTI,4.0,NaN,...,103873,160659,872,1549510.376,268549.322,101906.02,55498.17,17502.654,9832.638,0


In [11]:
df2['DISPO_CURR']= df2.DISPO_86032.where(cond=(df2.CV_NAME == '86032'),other=df2['DISPO_CURR'])
df2['DISPO_CURR']= df2.DISPO_91010.where(cond=(df2.CV_NAME == '91010'),other=df2['DISPO_CURR'])
df2['DISPO_CURR']= df2['DISPO_M 265'].where(cond=(df2.CV_NAME == 'M 265'),other=df2['DISPO_CURR'])
df2['DISPO_CURR']= df2['DISPO_SNK-09293'].where(cond=(df2.CV_NAME == 'SNK-09293'),other=df2['DISPO_CURR'])
df2['DISPO_CURR']= df2['DISPO_PI 00-1110'].where(cond=(df2.CV_NAME == 'PI 00-1110'),other=df2['DISPO_CURR'])
df2['DISPO_CURR']= df2['DISPO_CO VSI 08005'].where(cond=(df2.CV_NAME == 'CO VSI 08005'),other=df2['DISPO_CURR'])
df2

,SEASON_CD,SEASON_NAME,PLOT_CLUSTER_CD,CLUSTER_NAME,PLOT_VILLAGE_CD,VILLAGE_NAME,FARMER_ID,F_NAME,PLOT_NO,AG_FLAG,...,GBL_KHATE_ID,GBL_PLOT_ID,DISPOSAL_TIME,DISPO_86032,DISPO_91010,DISPO_M 265,DISPO_SNK-09293,DISPO_PI 00-1110,DISPO_CO VSI 08005,DISPO_CURR
35122,25,2019-2020,9,MUDHOL-SE,312,CHIKKUR,109725,ASHOK RAMAPPA GALAGALI,3.0,NaN,...,19971,37526,4,0.000,0.000,0.00,0.00,0.000,0.000,0.000
47629,25,2019-2020,20,SHIRAGUPPI,909,JAMAKHANDI-N,144756,MURAGEPPA BASAPPA KOUJALAGI,92.0,NaN,...,225147,311815,14,0.000,0.000,0.00,0.00,0.000,0.000,0.000
12481,25,2019-2020,2,FACTORY-SW,900,KAPPALAGUDDI-N,364757,ALLAPPA SHANMUK ANGADI,1.0,NaN,...,232595,320753,16,0.000,0.000,0.00,0.00,0.000,0.000,0.000
825,25,2019-2020,1,FACTORY EAST,4,BELAGALI,100820,SHIDAGIREPPA MAHALINGAPP BELAGALI,6.0,NaN,...,1424,2676,19,0.000,0.000,0.00,0.00,0.000,0.000,0.000
9327,25,2019-2020,2,FACTORY-SW,18,SHIVAPUR,301042,MAHADEV SHIVABASU SILANNAVAR,1.0,NaN,...,146630,218730,40,0.000,0.000,0.00,0.00,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1330,25,2019-2020,1,FACTORY EAST,4,BELAGALI,349833,SATAWWA CHANNAYYA CHATTI,3.0,NaN,...,192604,273245,870,1549354.041,268549.322,101906.02,55498.17,17502.654,9832.638,1549354.041
7946,25,2019-2020,2,FACTORY-SW,16,MARAKUDI,151027,VITHAL TULAJAPPA MALAGAR,4.0,NaN,...,250324,341527,872,1549378.621,268549.322,101906.02,55498.17,17502.654,9832.638,1549378.621
7883,25,2019-2020,2,FACTORY-SW,16,MARAKUDI,133379,BASAPPA KALLAPPA KULIGOD,1.0,NaN,...,80333,126982,872,1549456.526,268549.322,101906.02,55498.17,17502.654,9832.638,1549456.526
12058,25,2019-2020,2,FACTORY-SW,900,KAPPALAGUDDI-N,143662,SADAPPA TUKKAPPA BHADRASHETTI,4.0,NaN,...,103873,160659,872,1549510.376,268549.322,101906.02,55498.17,17502.654,9832.638,1549510.376


In [12]:
X2=df2[["CV_NAME","CLUSTER_NAME","PLANT_AREA","CROP_TYPE_NAME","AGE_IN_DAYS","DISPOSAL_TIME","DISPO_CURR"]].copy()
Y2=df2["DIVERSION_CENTER_NAME"].copy()

for i in Y2.index:
    Y2[i]=(1 if (Y2[i]=="GSM_HARVEST") else 0)
Y2_new=Y2.astype('int')

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

features = ["CLUSTER_NAME","CROP_TYPE_NAME","CV_NAME"]
one = OneHotEncoder(sparse=False)
transformer = ColumnTransformer([("one",
                                  one,
                                  features)],
                                  remainder="passthrough")
X2_new = pd.DataFrame(transformer.fit_transform(X2))
X2_new,Y2_new

(        0    1    2    3    4    5    6    7    8    9   ...   23   24   25  \
 0      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 1      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 2      0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 3      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 4      0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 ...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
 52418  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 52419  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 52420  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 52421  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 52422  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 
         26   27   28   29     30     

In [16]:
X2train,X2test,Y2train,Y2test =train_test_split(X2_new,Y2_new,test_size=0.2)
def score2(model):
    model.fit(X2train,Y2train)
    print(model.score(X2test,Y2test))

In [17]:
score2(rf)
score2(lr)
score2(kn)
score2(rc)
score2(svc)
score2(bc)
score2(gbc)
score2(hgbc)
score2(lgbm)
score2(xgboost)
score2(catboost)

0.9112064854554125
0.7760610395803529
0.7899856938483548
0.8225083452551264


C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=9.23766e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,


0.6778254649499285
0.8981402002861231
0.892894611349547
0.907105388650453
0.9081545064377683
[10:40:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9134000953743443
Learning rate set to 0.050791
0:	learn: 0.6271348	total: 11.3ms	remaining: 11.3s
1:	learn: 0.5741810	total: 22.8ms	remaining: 11.4s
2:	learn: 0.5301452	total: 33.6ms	remaining: 11.2s
3:	learn: 0.4921540	total: 43.8ms	remaining: 10.9s
4:	learn: 0.4635422	total: 54.2ms	remaining: 10.8s
5:	learn: 0.4380403	total: 64.8ms	remaining: 10.7s
6:	learn: 0.4227861	total: 75.1ms	remaining: 10.7s
7:	learn: 0.4083461	total: 85.6ms	remaining: 10.6s
8:	learn: 0.3980144	total: 95.8ms	remaining: 10.5s
9:	learn: 0.3855696	total: 106ms	remaining: 10.5s
10:	learn: 0.3733510	total: 116ms	remai

154:	learn: 0.2419609	total: 1.67s	remaining: 9.11s
155:	learn: 0.2417925	total: 1.68s	remaining: 9.11s
156:	learn: 0.2415797	total: 1.7s	remaining: 9.1s
157:	learn: 0.2414722	total: 1.71s	remaining: 9.09s
158:	learn: 0.2412883	total: 1.72s	remaining: 9.08s
159:	learn: 0.2411705	total: 1.73s	remaining: 9.06s
160:	learn: 0.2410439	total: 1.74s	remaining: 9.05s
161:	learn: 0.2408682	total: 1.75s	remaining: 9.04s
162:	learn: 0.2406948	total: 1.76s	remaining: 9.03s
163:	learn: 0.2404487	total: 1.77s	remaining: 9.02s
164:	learn: 0.2403287	total: 1.78s	remaining: 9s
165:	learn: 0.2401160	total: 1.79s	remaining: 8.99s
166:	learn: 0.2399283	total: 1.8s	remaining: 8.97s
167:	learn: 0.2398372	total: 1.81s	remaining: 8.96s
168:	learn: 0.2396902	total: 1.82s	remaining: 8.95s
169:	learn: 0.2395893	total: 1.83s	remaining: 8.94s
170:	learn: 0.2394813	total: 1.84s	remaining: 8.93s
171:	learn: 0.2391769	total: 1.85s	remaining: 8.91s
172:	learn: 0.2391149	total: 1.86s	remaining: 8.9s
173:	learn: 0.23892

328:	learn: 0.2202591	total: 3.6s	remaining: 7.35s
329:	learn: 0.2201237	total: 3.62s	remaining: 7.34s
330:	learn: 0.2200342	total: 3.63s	remaining: 7.34s
331:	learn: 0.2199426	total: 3.64s	remaining: 7.33s
332:	learn: 0.2198107	total: 3.65s	remaining: 7.32s
333:	learn: 0.2197844	total: 3.67s	remaining: 7.31s
334:	learn: 0.2197080	total: 3.68s	remaining: 7.3s
335:	learn: 0.2195957	total: 3.69s	remaining: 7.29s
336:	learn: 0.2194891	total: 3.7s	remaining: 7.28s
337:	learn: 0.2194148	total: 3.71s	remaining: 7.27s
338:	learn: 0.2193031	total: 3.72s	remaining: 7.26s
339:	learn: 0.2192154	total: 3.73s	remaining: 7.25s
340:	learn: 0.2191926	total: 3.74s	remaining: 7.23s
341:	learn: 0.2191102	total: 3.75s	remaining: 7.22s
342:	learn: 0.2190248	total: 3.76s	remaining: 7.21s
343:	learn: 0.2188781	total: 3.77s	remaining: 7.2s
344:	learn: 0.2187521	total: 3.78s	remaining: 7.18s
345:	learn: 0.2186293	total: 3.79s	remaining: 7.17s
346:	learn: 0.2185652	total: 3.81s	remaining: 7.16s
347:	learn: 0.21

501:	learn: 0.2073429	total: 5.54s	remaining: 5.5s
502:	learn: 0.2073279	total: 5.55s	remaining: 5.49s
503:	learn: 0.2072398	total: 5.57s	remaining: 5.48s
504:	learn: 0.2071588	total: 5.58s	remaining: 5.47s
505:	learn: 0.2070763	total: 5.59s	remaining: 5.46s
506:	learn: 0.2069876	total: 5.6s	remaining: 5.45s
507:	learn: 0.2069710	total: 5.61s	remaining: 5.44s
508:	learn: 0.2069139	total: 5.62s	remaining: 5.42s
509:	learn: 0.2068484	total: 5.64s	remaining: 5.42s
510:	learn: 0.2068026	total: 5.65s	remaining: 5.41s
511:	learn: 0.2067177	total: 5.66s	remaining: 5.4s
512:	learn: 0.2066495	total: 5.67s	remaining: 5.38s
513:	learn: 0.2066003	total: 5.68s	remaining: 5.37s
514:	learn: 0.2065412	total: 5.69s	remaining: 5.36s
515:	learn: 0.2065040	total: 5.7s	remaining: 5.35s
516:	learn: 0.2064397	total: 5.71s	remaining: 5.34s
517:	learn: 0.2063601	total: 5.72s	remaining: 5.33s
518:	learn: 0.2062616	total: 5.74s	remaining: 5.32s
519:	learn: 0.2062301	total: 5.75s	remaining: 5.31s
520:	learn: 0.20

671:	learn: 0.1979876	total: 7.47s	remaining: 3.65s
672:	learn: 0.1979353	total: 7.49s	remaining: 3.64s
673:	learn: 0.1978601	total: 7.5s	remaining: 3.63s
674:	learn: 0.1977873	total: 7.51s	remaining: 3.62s
675:	learn: 0.1977164	total: 7.52s	remaining: 3.6s
676:	learn: 0.1976569	total: 7.53s	remaining: 3.59s
677:	learn: 0.1976251	total: 7.54s	remaining: 3.58s
678:	learn: 0.1975746	total: 7.55s	remaining: 3.57s
679:	learn: 0.1975224	total: 7.56s	remaining: 3.56s
680:	learn: 0.1974808	total: 7.58s	remaining: 3.55s
681:	learn: 0.1974298	total: 7.59s	remaining: 3.54s
682:	learn: 0.1973408	total: 7.6s	remaining: 3.52s
683:	learn: 0.1972860	total: 7.61s	remaining: 3.51s
684:	learn: 0.1972091	total: 7.62s	remaining: 3.5s
685:	learn: 0.1971630	total: 7.63s	remaining: 3.49s
686:	learn: 0.1971442	total: 7.64s	remaining: 3.48s
687:	learn: 0.1970909	total: 7.65s	remaining: 3.47s
688:	learn: 0.1970346	total: 7.66s	remaining: 3.46s
689:	learn: 0.1969520	total: 7.67s	remaining: 3.45s
690:	learn: 0.19

845:	learn: 0.1901278	total: 9.42s	remaining: 1.71s
846:	learn: 0.1900694	total: 9.43s	remaining: 1.7s
847:	learn: 0.1900458	total: 9.44s	remaining: 1.69s
848:	learn: 0.1900021	total: 9.45s	remaining: 1.68s
849:	learn: 0.1899451	total: 9.46s	remaining: 1.67s
850:	learn: 0.1899095	total: 9.48s	remaining: 1.66s
851:	learn: 0.1898799	total: 9.49s	remaining: 1.65s
852:	learn: 0.1898258	total: 9.5s	remaining: 1.64s
853:	learn: 0.1898109	total: 9.51s	remaining: 1.63s
854:	learn: 0.1897916	total: 9.52s	remaining: 1.61s
855:	learn: 0.1897515	total: 9.53s	remaining: 1.6s
856:	learn: 0.1897284	total: 9.54s	remaining: 1.59s
857:	learn: 0.1896871	total: 9.56s	remaining: 1.58s
858:	learn: 0.1896447	total: 9.57s	remaining: 1.57s
859:	learn: 0.1896083	total: 9.58s	remaining: 1.56s
860:	learn: 0.1895600	total: 9.59s	remaining: 1.55s
861:	learn: 0.1895287	total: 9.6s	remaining: 1.54s
862:	learn: 0.1894783	total: 9.61s	remaining: 1.53s
863:	learn: 0.1894438	total: 9.63s	remaining: 1.51s
864:	learn: 0.18

In [13]:
one=pd.DataFrame({'a':df2.DISPO_CURR})
two = pd.DataFrame({"a":[0]}).copy()
two=two.append(one)
two=two.reset_index(drop=True)
two.tail(10),one.tail(10)


(                 a
 52414  1549091.513
 52415  1549135.098
 52416  1549248.912
 52417  1549276.461
 52418  1549319.231
 52419  1549354.041
 52420  1549378.621
 52421  1549456.526
 52422  1549510.376
 52423  1549538.106,
                  a
 3272   1549091.513
 5887   1549135.098
 3230   1549248.912
 3189   1549276.461
 1343   1549319.231
 1330   1549354.041
 7946   1549378.621
 7883   1549456.526
 12058  1549510.376
 7534   1549538.106)

In [14]:
df3=df2.reset_index(drop=True).copy()
df3['DISPO_CURR2']=two['a']
df3

,SEASON_CD,SEASON_NAME,PLOT_CLUSTER_CD,CLUSTER_NAME,PLOT_VILLAGE_CD,VILLAGE_NAME,FARMER_ID,F_NAME,PLOT_NO,AG_FLAG,...,GBL_PLOT_ID,DISPOSAL_TIME,DISPO_86032,DISPO_91010,DISPO_M 265,DISPO_SNK-09293,DISPO_PI 00-1110,DISPO_CO VSI 08005,DISPO_CURR,DISPO_CURR2
0,25,2019-2020,9,MUDHOL-SE,312,CHIKKUR,109725,ASHOK RAMAPPA GALAGALI,3.0,NaN,...,37526,4,0.000,0.000,0.00,0.00,0.000,0.000,0.000,0.000
1,25,2019-2020,20,SHIRAGUPPI,909,JAMAKHANDI-N,144756,MURAGEPPA BASAPPA KOUJALAGI,92.0,NaN,...,311815,14,0.000,0.000,0.00,0.00,0.000,0.000,0.000,0.000
2,25,2019-2020,2,FACTORY-SW,900,KAPPALAGUDDI-N,364757,ALLAPPA SHANMUK ANGADI,1.0,NaN,...,320753,16,0.000,0.000,0.00,0.00,0.000,0.000,0.000,0.000
3,25,2019-2020,1,FACTORY EAST,4,BELAGALI,100820,SHIDAGIREPPA MAHALINGAPP BELAGALI,6.0,NaN,...,2676,19,0.000,0.000,0.00,0.00,0.000,0.000,0.000,0.000
4,25,2019-2020,2,FACTORY-SW,18,SHIVAPUR,301042,MAHADEV SHIVABASU SILANNAVAR,1.0,NaN,...,218730,40,0.000,0.000,0.00,0.00,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52418,25,2019-2020,1,FACTORY EAST,4,BELAGALI,349833,SATAWWA CHANNAYYA CHATTI,3.0,NaN,...,273245,870,1549354.041,268549.322,101906.02,55498.17,17502.654,9832.638,1549354.041,1549319.231
52419,25,2019-2020,2,FACTORY-SW,16,MARAKUDI,151027,VITHAL TULAJAPPA MALAGAR,4.0,NaN,...,341527,872,1549378.621,268549.322,101906.02,55498.17,17502.654,9832.638,1549378.621,1549354.041
52420,25,2019-2020,2,FACTORY-SW,16,MARAKUDI,133379,BASAPPA KALLAPPA KULIGOD,1.0,NaN,...,126982,872,1549456.526,268549.322,101906.02,55498.17,17502.654,9832.638,1549456.526,1549378.621
52421,25,2019-2020,2,FACTORY-SW,900,KAPPALAGUDDI-N,143662,SADAPPA TUKKAPPA BHADRASHETTI,4.0,NaN,...,160659,872,1549510.376,268549.322,101906.02,55498.17,17502.654,9832.638,1549510.376,1549456.526


In [15]:
X3=df3[["CV_NAME","CLUSTER_NAME","PLANT_AREA","CROP_TYPE_NAME","AGE_IN_DAYS","DISPOSAL_TIME","DISPO_CURR2"]].copy()
Y3=df3["DIVERSION_CENTER_NAME"].copy()

for i in Y3.index:
    Y3[i]=(1 if (Y3[i]=="GSM_HARVEST") else 0)
Y3_new=Y3.astype('int')

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

features = ["CLUSTER_NAME","CROP_TYPE_NAME","CV_NAME"]
one = OneHotEncoder(sparse=False)
transformer = ColumnTransformer([("one",
                                  one,
                                  features)],
                                  remainder="passthrough")
X3_new = pd.DataFrame(transformer.fit_transform(X3))
X3_new,Y3_new

(        0    1    2    3    4    5    6    7    8    9   ...   23   24   25  \
 0      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 1      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 2      0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 3      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 4      0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 ...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
 52418  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 52419  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 52420  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 52421  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 52422  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
 
         26   27   28   29     30     

In [86]:
X3train,X3test,Y3train,Y3test =train_test_split(X3_new,Y3_new,test_size=0.2)
def score3(model):
    model.fit(X3train,Y3train)
    print(model.score(X3test,Y3test))

In [87]:
score3(rf)
score3(lr)
score3(kn)
score3(rc)
score3(svc)
score3(bc)
score3(gbc)
score3(hgbc)
score3(lgbm)
score3(xgboost)
score3(catboost)

0.9023366714353839
0.7598474010491177
0.7852169766332856
0.8103004291845494


C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=9.2894e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,


0.6659036719122556
0.8985216976633286
0.8909871244635194
0.9050071530758226
0.9045302813543157
[11:34:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9092989985693848
Learning rate set to 0.050791
0:	learn: 0.6264234	total: 12.5ms	remaining: 12.5s
1:	learn: 0.5673666	total: 25ms	remaining: 12.5s
2:	learn: 0.5248128	total: 38.7ms	remaining: 12.8s
3:	learn: 0.4881754	total: 52.9ms	remaining: 13.2s
4:	learn: 0.4598540	total: 66.9ms	remaining: 13.3s
5:	learn: 0.4349240	total: 85.2ms	remaining: 14.1s
6:	learn: 0.4172917	total: 99.9ms	remaining: 14.2s
7:	learn: 0.4005537	total: 116ms	remaining: 14.4s
8:	learn: 0.3907007	total: 134ms	remaining: 14.7s
9:	learn: 0.3787081	total: 145ms	remaining: 14.4s
10:	learn: 0.3709804	total: 158ms	remaini

162:	learn: 0.2390209	total: 2.12s	remaining: 10.9s
163:	learn: 0.2388689	total: 2.13s	remaining: 10.9s
164:	learn: 0.2387369	total: 2.15s	remaining: 10.9s
165:	learn: 0.2386171	total: 2.16s	remaining: 10.9s
166:	learn: 0.2384699	total: 2.17s	remaining: 10.9s
167:	learn: 0.2382487	total: 2.19s	remaining: 10.8s
168:	learn: 0.2381368	total: 2.2s	remaining: 10.8s
169:	learn: 0.2379864	total: 2.22s	remaining: 10.8s
170:	learn: 0.2378241	total: 2.23s	remaining: 10.8s
171:	learn: 0.2376851	total: 2.25s	remaining: 10.8s
172:	learn: 0.2374908	total: 2.26s	remaining: 10.8s
173:	learn: 0.2372807	total: 2.27s	remaining: 10.8s
174:	learn: 0.2372038	total: 2.28s	remaining: 10.8s
175:	learn: 0.2371105	total: 2.29s	remaining: 10.7s
176:	learn: 0.2369767	total: 2.31s	remaining: 10.7s
177:	learn: 0.2367199	total: 2.32s	remaining: 10.7s
178:	learn: 0.2365350	total: 2.33s	remaining: 10.7s
179:	learn: 0.2364178	total: 2.35s	remaining: 10.7s
180:	learn: 0.2362361	total: 2.36s	remaining: 10.7s
181:	learn: 0

322:	learn: 0.2183214	total: 4.09s	remaining: 8.58s
323:	learn: 0.2182505	total: 4.11s	remaining: 8.57s
324:	learn: 0.2181723	total: 4.12s	remaining: 8.55s
325:	learn: 0.2180897	total: 4.13s	remaining: 8.54s
326:	learn: 0.2179952	total: 4.14s	remaining: 8.52s
327:	learn: 0.2179064	total: 4.15s	remaining: 8.51s
328:	learn: 0.2178262	total: 4.16s	remaining: 8.49s
329:	learn: 0.2177789	total: 4.18s	remaining: 8.48s
330:	learn: 0.2177115	total: 4.19s	remaining: 8.47s
331:	learn: 0.2176390	total: 4.2s	remaining: 8.46s
332:	learn: 0.2175357	total: 4.21s	remaining: 8.44s
333:	learn: 0.2175113	total: 4.22s	remaining: 8.43s
334:	learn: 0.2173816	total: 4.24s	remaining: 8.41s
335:	learn: 0.2172918	total: 4.25s	remaining: 8.4s
336:	learn: 0.2172765	total: 4.26s	remaining: 8.38s
337:	learn: 0.2171874	total: 4.27s	remaining: 8.37s
338:	learn: 0.2171172	total: 4.28s	remaining: 8.35s
339:	learn: 0.2170433	total: 4.3s	remaining: 8.34s
340:	learn: 0.2169556	total: 4.31s	remaining: 8.33s
341:	learn: 0.2

492:	learn: 0.2062707	total: 6.02s	remaining: 6.19s
493:	learn: 0.2061775	total: 6.04s	remaining: 6.18s
494:	learn: 0.2061070	total: 6.05s	remaining: 6.17s
495:	learn: 0.2060652	total: 6.06s	remaining: 6.16s
496:	learn: 0.2060009	total: 6.07s	remaining: 6.14s
497:	learn: 0.2058994	total: 6.08s	remaining: 6.13s
498:	learn: 0.2058330	total: 6.09s	remaining: 6.12s
499:	learn: 0.2057490	total: 6.1s	remaining: 6.1s
500:	learn: 0.2056930	total: 6.11s	remaining: 6.09s
501:	learn: 0.2056288	total: 6.12s	remaining: 6.07s
502:	learn: 0.2055565	total: 6.13s	remaining: 6.06s
503:	learn: 0.2054820	total: 6.14s	remaining: 6.05s
504:	learn: 0.2053911	total: 6.16s	remaining: 6.03s
505:	learn: 0.2053433	total: 6.17s	remaining: 6.02s
506:	learn: 0.2053061	total: 6.18s	remaining: 6.01s
507:	learn: 0.2052007	total: 6.19s	remaining: 5.99s
508:	learn: 0.2051159	total: 6.2s	remaining: 5.98s
509:	learn: 0.2050993	total: 6.21s	remaining: 5.96s
510:	learn: 0.2050657	total: 6.22s	remaining: 5.95s
511:	learn: 0.2

653:	learn: 0.1969602	total: 7.78s	remaining: 4.12s
654:	learn: 0.1969086	total: 7.79s	remaining: 4.1s
655:	learn: 0.1968666	total: 7.8s	remaining: 4.09s
656:	learn: 0.1967930	total: 7.81s	remaining: 4.08s
657:	learn: 0.1967784	total: 7.82s	remaining: 4.07s
658:	learn: 0.1967285	total: 7.83s	remaining: 4.05s
659:	learn: 0.1966842	total: 7.84s	remaining: 4.04s
660:	learn: 0.1966211	total: 7.86s	remaining: 4.03s
661:	learn: 0.1965688	total: 7.87s	remaining: 4.02s
662:	learn: 0.1965222	total: 7.88s	remaining: 4.01s
663:	learn: 0.1964625	total: 7.89s	remaining: 3.99s
664:	learn: 0.1964481	total: 7.91s	remaining: 3.98s
665:	learn: 0.1964273	total: 7.92s	remaining: 3.97s
666:	learn: 0.1963889	total: 7.93s	remaining: 3.96s
667:	learn: 0.1963560	total: 7.94s	remaining: 3.95s
668:	learn: 0.1963285	total: 7.95s	remaining: 3.94s
669:	learn: 0.1962893	total: 7.96s	remaining: 3.92s
670:	learn: 0.1962265	total: 7.98s	remaining: 3.91s
671:	learn: 0.1961454	total: 7.99s	remaining: 3.9s
672:	learn: 0.1

822:	learn: 0.1890998	total: 9.75s	remaining: 2.1s
823:	learn: 0.1890773	total: 9.76s	remaining: 2.08s
824:	learn: 0.1890365	total: 9.77s	remaining: 2.07s
825:	learn: 0.1889924	total: 9.78s	remaining: 2.06s
826:	learn: 0.1889334	total: 9.79s	remaining: 2.05s
827:	learn: 0.1888889	total: 9.8s	remaining: 2.04s
828:	learn: 0.1888299	total: 9.81s	remaining: 2.02s
829:	learn: 0.1887828	total: 9.82s	remaining: 2.01s
830:	learn: 0.1887750	total: 9.83s	remaining: 2s
831:	learn: 0.1887343	total: 9.84s	remaining: 1.99s
832:	learn: 0.1886880	total: 9.85s	remaining: 1.98s
833:	learn: 0.1886750	total: 9.86s	remaining: 1.96s
834:	learn: 0.1886383	total: 9.87s	remaining: 1.95s
835:	learn: 0.1885945	total: 9.88s	remaining: 1.94s
836:	learn: 0.1885547	total: 9.89s	remaining: 1.93s
837:	learn: 0.1885162	total: 9.9s	remaining: 1.91s
838:	learn: 0.1884679	total: 9.91s	remaining: 1.9s
839:	learn: 0.1884200	total: 9.92s	remaining: 1.89s
840:	learn: 0.1883591	total: 9.94s	remaining: 1.88s
841:	learn: 0.18831

985:	learn: 0.1826437	total: 11.5s	remaining: 163ms
986:	learn: 0.1825872	total: 11.5s	remaining: 151ms
987:	learn: 0.1825401	total: 11.5s	remaining: 139ms
988:	learn: 0.1824951	total: 11.5s	remaining: 128ms
989:	learn: 0.1824472	total: 11.5s	remaining: 116ms
990:	learn: 0.1824147	total: 11.5s	remaining: 105ms
991:	learn: 0.1823687	total: 11.5s	remaining: 92.9ms
992:	learn: 0.1823257	total: 11.5s	remaining: 81.3ms
993:	learn: 0.1822893	total: 11.5s	remaining: 69.7ms
994:	learn: 0.1822640	total: 11.6s	remaining: 58ms
995:	learn: 0.1822518	total: 11.6s	remaining: 46.4ms
996:	learn: 0.1822348	total: 11.6s	remaining: 34.8ms
997:	learn: 0.1822255	total: 11.6s	remaining: 23.2ms
998:	learn: 0.1821981	total: 11.6s	remaining: 11.6ms
999:	learn: 0.1821721	total: 11.6s	remaining: 0us
0.9085360038149738


In [16]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
def cv_score(model,X,Y):
    cv = KFold(n_splits=10, random_state=1, shuffle=True)
    scores = cross_val_score(model, X, Y, scoring='accuracy', cv=cv, n_jobs=-1)
    print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

In [17]:
cv_score(catboost,X1_new,Y1_new)
cv_score(catboost,X2_new,Y2_new)
cv_score(catboost,X2_new,Y2_new)

Accuracy: 0.912 (0.004)
Accuracy: 0.911 (0.003)
Accuracy: 0.911 (0.003)


In [98]:
cv_score(rf,X1_new,Y1_new)
cv_score(rf,X2_new,Y2_new)
cv_score(rf,X2_new,Y2_new)
cv_score(xgboost,X1_new,Y1_new)
cv_score(xgboost,X2_new,Y2_new)
cv_score(xgboost,X2_new,Y2_new)
cv_score(lgbm,X1_new,Y1_new)
cv_score(lgbm,X2_new,Y2_new)
cv_score(lgbm,X2_new,Y2_new)
cv_score(hgbc,X1_new,Y1_new)
cv_score(hgbc,X2_new,Y2_new)
cv_score(hgbc,X2_new,Y2_new)

Accuracy: 0.900 (0.003)
Accuracy: 0.907 (0.005)
Accuracy: 0.906 (0.005)
Accuracy: 0.911 (0.005)
Accuracy: 0.910 (0.003)
Accuracy: 0.910 (0.003)
Accuracy: 0.907 (0.005)
Accuracy: 0.907 (0.003)
Accuracy: 0.907 (0.003)
Accuracy: 0.906 (0.005)
Accuracy: 0.906 (0.003)
Accuracy: 0.906 (0.004)


In [18]:
X1_new,X2_new,X3_new

(        0    1    2    3    4    5    6    7    8    9   ...   22   23   24  \
 0      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  1.0   
 1      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  1.0   
 2      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  1.0   
 3      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  1.0  0.0   
 4      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 ...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
 52418  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  1.0  0.0   
 52419  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  1.0  0.0   
 52420  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0  1.0   
 52421  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0  1.0   
 52422  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  1.0  0.0   
 
         25   26   27   28   29     30

In [19]:
df.AGE_IN_DAYS.min()

-734067

In [23]:
df.CV_NAME.value_counts()

86032           37802
91010            9001
M 265            4305
SNK-09293         922
PI 00-1110        434
CO VSI 08005      311
CO 92005           95
CO-2001-13         91
MS-10001           88
OTHERS             70
8011               63
671                59
SNK-09227          43
CO-09004           41
CO SNK 07680       38
CO62175            28
CO-SNK-632         11
MIX                 6
94012               5
CO-2001-15          3
CO-15017            3
CO-14031            2
92020               2
EC-14006            1
CO-2012-293         1
Name: CV_NAME, dtype: int64